In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
#training = pd.read_csv('C:/Users/Alvis/STA142A/train.csv')
#test = pd.read_csv('C:/Users/Alvis/STA142A/test.csv')
#test_labels = pd.read_csv("C:/Users/Alvis/STA142A/test_labels.csv")

In [ ]:
#training = pd.read_csv("C:/Users/Falak/Desktop/NLP/train.csv")
#test = pd.read_csv("C:/Users/Falak/Desktop/NLP/test.csv")
#test_labels = pd.read_csv("C:/Users/Falak/Desktop/NLP/test_labels.csv")

In [ ]:
training = pd.read_csv('C:/Users/dlsim/Documents/_College/1 UC Davis/4 Senior Year/Win - STA 142a - Statistical Learning I/142 Final Project/train.csv')
test = pd.read_csv('C:/Users/dlsim/Documents/_College/1 UC Davis/4 Senior Year/Win - STA 142a - Statistical Learning I/142 Final Project/test.csv')
test_labels = pd.read_csv('C:/Users/dlsim/Documents/_College/1 UC Davis/4 Senior Year/Win - STA 142a - Statistical Learning I/142 Final Project/test_labels.csv')

In [ ]:
# Observing data
print(f"Train Data dimensions are {training.shape}")
#print(f"Train Data dimensions are {test.shape}")

training.head(3)

In [ ]:
# Visualization - https://www.kaggle.com/sasidharturaga/eda-step-wise-preprocess-and-lstm-classifier
import plotly.express as px

df1 = pd.DataFrame(training[training.columns[2:]].sum(axis=0)).reset_index() 
df1["Type of Comment Violation"] = df1["index"]
df1["Number of Comments"] = df1[0]
df1.head(6)

In [ ]:
df2 = pd.DataFrame(pd.DataFrame(training[training.columns[2:]].sum(axis=1)).reset_index()[0].value_counts())
df2["Number of Sentences"]=df2[0]
df2["Number of Types of Violations"]=df2.index
df2

In [ ]:
#Display of comments 
df1 = df1.sort_values(0, ascending=False) 
fig1 = px.bar(df1, x="Type of Comment Violation", y="Number of Comments", title="No. of comments per label",text="Number of Comments")
fig1.show()

fig2 = px.bar(df2, x="Number of Types of Violations", y="Number of Sentences", title="Number of Comments by Amount of Labels", text="Number of Sentences")
fig2.show()